In [12]:
####
#Author: brandon chiazza
#version 1.0
#references: 
#https://www.programiz.com/python-programming/working-csv-files
#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
#https://realpython.com/python-boto3-aws-s3/
#CLI aws s3api create-bucket --bucket my-bucket-name --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2
#https://robertorocha.info/setting-up-a-selenium-web-scraper-on-aws-lambda-with-python/ 
##

###Load libraries
import awscli
import selenium
import boto3
import pandas as pd
import time

from selenium import webdriver


####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome("C:/Program Files (x86)/Microsoft Visual Studio/Shared/Python37_64/chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)


<selenium.webdriver.remote.webelement.WebElement (session="13ac90544701a6e83acf8633bb97114f", element="78101d62-49de-4aca-92dc-4f1fde76e40a")>


In [13]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3 

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
2,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
3,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
4,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
5,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
6,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
7,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
8,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY
9,1000 Islands Hose Haulers,45-38-38,454570241,NFP,CARTHAGE,NY


In [14]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/NAME_OF_YOUR_S3_BUCKET/'#specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/database-update-bucket/charities_bureau_scrape_20200725151624.csv
